<a href="https://colab.research.google.com/github/ShreyaRaundal/1/blob/main/gen4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Music generation with RNN
#link used: https://bitmidi.com

# Step 1: Install Required Libraries
!pip install pretty_midi
!pip install tensorflow

# Step 2: Import Libraries
import os
import pretty_midi
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Step 3: Create Folder and Upload MIDI Files (Colab only)
from google.colab import files

midi_folder = "/content/midi/"
os.makedirs(midi_folder, exist_ok=True)

print("Upload MIDI files (with .mid extension)")
uploaded = files.upload()
for filename in uploaded:
    os.rename(filename, os.path.join(midi_folder, filename))

# Step 4: Load and Parse MIDI Files
notes = []
for file in os.listdir(midi_folder):
    if file.endswith(".mid"):
        midi = pretty_midi.PrettyMIDI(os.path.join(midi_folder, file))
        for instrument in midi.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    notes.append(note.pitch)

print("Total notes collected:", len(notes))

# Step 5: Preprocess the Note Sequences
encoder = LabelEncoder()
encoded_notes = encoder.fit_transform(notes)

seq_length = 50
sequences = []
targets = []
for i in range(len(encoded_notes) - seq_length):
    sequences.append(encoded_notes[i:i + seq_length])
    targets.append(encoded_notes[i + seq_length])

X = np.array(sequences)
y = np.array(targets)
y = to_categorical(y, num_classes=len(encoder.classes_))

# Step 6: Build the RNN Model
model = Sequential([
    Embedding(input_dim=len(encoder.classes_), output_dim=100, input_length=seq_length),
    LSTM(256, return_sequences=True),
    LSTM(256),
    Dense(len(encoder.classes_), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

# Step 7: Train the Model
model.fit(X, y, batch_size=64, epochs=2)

# Step 8: Generate New Music
def generate_music(model, encoder, seed_sequence, length=200):
    generated_notes = seed_sequence[:]
    for _ in range(length):
        input_sequence = np.array([generated_notes[-50:]])
        predicted_probabilities = model.predict(input_sequence, verbose=0)
        predicted_note = np.argmax(predicted_probabilities)
        generated_notes.append(predicted_note)
    return encoder.inverse_transform(generated_notes)

# Step 9: Generate and Print Notes
seed_sequence = list(encoded_notes[:50])
generated_notes = generate_music(model, encoder, seed_sequence, length=200)
print("Generated Notes:", generated_notes)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=e167b884976d76601fb4b7dd8e9db50721042f1c14095fe74ba76100e65f0d95
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
Upload MIDI files (with .mid extension)


Saving Queen - Bohemian Rhapsody.mid to Queen - Bohemian Rhapsody.mid
Total notes collected: 4811


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
47/75 ━━━━━━━━━━━━━━━━━━━━ 16s 597ms/step - loss: 3.8920